In [20]:
import pandas as pd
from datetime import datetime, timedelta

# Load the datasets
plotdata_df = pd.read_csv('../5_processing_extracted_data/plotdata.csv', usecols=['date', 'TH/J'])
plotdata_df['date'] = pd.to_datetime(plotdata_df['date']).dt.date
max_efficiency_df = pd.read_csv('../../hardwarelist/Bitcoin max updated2.csv', usecols=['date', 'max (TH/J)'])
max_efficiency_df['date'] = pd.to_datetime(max_efficiency_df['date']).dt.date
price_df = pd.read_csv('../../pricehistory/price_full.csv', usecols=['Start', 'Open'])
price_df['Start'] = pd.to_datetime(price_df['Start']).dt.date

# Process plotdata.csv to get average power efficiency per day
avg_efficiency = plotdata_df.groupby('date')['TH/J'].mean().reset_index()
avg_efficiency.columns = ['date', 'avg_efficiency']

# get number of entries per day
counts = plotdata_df['date'].value_counts().reset_index()

In [21]:
counts

,date,count
0,2013-06-09,22
1,2014-08-17,19
2,2013-06-07,18
3,2015-01-01,16
4,2013-07-18,16
...,...,...
1784,2012-05-08,1
1785,2013-11-01,1
1786,2013-11-02,1
1787,2012-05-02,1


In [22]:

# Forward fill to handle days without entries
all_dates = pd.date_range(start=avg_efficiency['date'].min(), end=avg_efficiency['date'].max(), freq='D')
all_dates_df = pd.DataFrame(all_dates, columns=['date'])
all_dates_df['date'] = all_dates_df['date'].dt.date  # Ensure 'date' is of type date

avg_efficiency = pd.merge(all_dates_df, avg_efficiency, on='date', how='left').fillna(method='ffill')

# Merge with max efficiency
max_efficiency_df = max_efficiency_df.rename(columns={'max (TH/J)': 'max_efficiency'})
merged_df = pd.merge(avg_efficiency, max_efficiency_df, on='date', how='left')

# Merge with open price
price_df = price_df.rename(columns={'Start': 'date', 'Open': 'open_price'})
merged_df = pd.merge(merged_df, price_df, on='date', how='left')

C:\Users\Timothe\AppData\Local\Temp\ipykernel_44380\3606577326.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  avg_efficiency = pd.merge(all_dates_df, avg_efficiency, on='date', how='left').fillna(method='ffill')


In [23]:
merged_df

,date,avg_efficiency,max_efficiency,open_price
0,2011-05-01,0.000001,0.000004,3.0331
1,2011-05-02,0.000001,0.000004,3.2000
2,2011-05-03,0.000001,0.000004,3.4100
3,2011-05-04,0.000001,0.000004,3.4061
4,2011-05-05,0.000001,0.000004,3.3330
...,...,...,...,...
4560,2023-10-25,0.041667,NaN,33948.1200
4561,2023-10-26,0.041667,NaN,34467.3400
4562,2023-10-27,0.041667,NaN,34132.8000
4563,2023-10-28,0.041667,NaN,33899.7000


In [24]:
# Merge with counts
merged_df = pd.merge(merged_df, counts, on='date', how='left')
merged_df['count'] = merged_df['count'].fillna(0)

# Fill NaN values for open_price with the most recent non-NaN value
merged_df['open_price'] = merged_df['open_price'].fillna(method='ffill')

# Save to inputs.csv
merged_df.to_csv('inputs.csv', index=False)

print("Merging complete. Data saved to inputs.csv.")

Merging complete. Data saved to inputs.csv.


C:\Users\Timothe\AppData\Local\Temp\ipykernel_44380\3333721936.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df['open_price'] = merged_df['open_price'].fillna(method='ffill')
